In [1]:
%matplotlib inline 

import six
import math
import time
import Quandl
import calendar
import warnings
import numpy as np
import pandas as pd
import seaborn as sb
import pylab as pylab
from docx import Document
from datetime import datetime

from matplotlib import colors
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.mplot3d import Axes3D

from scipy.interpolate import interp1d
import statsmodels.stats.stattools as stats_stattools
import statsmodels.tsa.stattools as tsa_stattools
import statsmodels.tsa.seasonal as tsa_seasonal
import statsmodels.api as sm 
import xgboost as xgb
from unbalanced_dataset import SMOTE

from sklearn import svm
from sklearn import metrics, cross_validation, linear_model, naive_bayes, neighbors, ensemble
from sklearn import feature_selection
from sklearn import decomposition
from sklearn import discriminant_analysis
from sklearn import preprocessing

import sys
from os import listdir
from os.path import isfile, join
from helpers import features_analysis, procces_stocks, data_manipulation, download_quandl_data, ml_dataset, classifier_utils, report_generator, Iteration, Stacking, Boosting

fig_size = [10, 6]
plt.rcParams["figure.figsize"] = fig_size
sb.set_style('darkgrid')
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
GOLD = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/GOLD.csv')
SILVER = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/SILVER.csv')
PLAT = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/PLAT.csv')
OIL_BRENT = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/OIL_BRENT.csv')

USD_GBP = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/USD_GBP.csv')
JPY_USD = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/JPY_USD.csv')
AUD_USD = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/AUD_USD.csv')

INDEX_DJIA = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_DJIA.csv')
INDEX_HSI = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_HSI.csv')
INDEX_IBEX = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_IBEX.csv')
INDEX_N225 = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_N225.csv')
INDEX_SP500 = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_SP500.csv')
INDEX_AXJO = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_AXJO.csv')
INDEX_FCHI = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_FCHI.csv')
INDEX_GDAXI = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_GDAXI.csv')

values_names = ['GOLD', 'SILVER', 'PLAT', 'OIL_BRENT', 'USD_GBP', 'JPY_USD', 'AUD_USD', 'DJIA', 'HSI', 'IBEX', 'N225', 'SP500', 'AXJO', 'FCHI', 'GDAXI']
values_dfs = [GOLD, SILVER, PLAT, OIL_BRENT, USD_GBP, JPY_USD, AUD_USD, INDEX_DJIA, INDEX_HSI, INDEX_IBEX, INDEX_N225, INDEX_SP500, INDEX_AXJO, INDEX_FCHI, INDEX_GDAXI]
values_cols = ['USD', 'Value', 'Open', 'Close', 'High', 'Low', 'Volume']
dict_dfs_cols = {}
dict_dfs_cols2 = {}

for index in range(len(values_names)):
    name = values_names[index]
    df = values_dfs[index]    
    cols = df.columns.values

    new_cols = [x for x in cols if x not in ['Date', 'USD', 'Value', 'Open', 'Close', 'High', 'Low', 'Volume']]    
    new_cols2 = [x for x in cols if x not in ['Date']]    

    dict_dfs_cols[name] = new_cols
    dict_dfs_cols2[name] = new_cols2

dataset = ml_dataset.generate_df_dataset(values_names, values_dfs, dict_dfs_cols)
dataset_all = ml_dataset.generate_df_dataset(values_names, values_dfs, dict_dfs_cols2)



#First 30 row
dataset = dataset[31:]
dataset = dataset.reset_index(drop=True)
#colsToShift = [col for col in dataset.columns if 'HSI' in col or'N225' in col or'AXJO' in col]
#dataset[colsToShift] = dataset[colsToShift].shift(-1)
#last_row = dataset.shape[0]-1
#dataset = dataset.drop(last_row, axis=0)  

dataset_all = dataset_all[31:]
dataset_all = dataset_all.reset_index(drop=True)
#dataset_all[colsToShift] = dataset_all[colsToShift].shift(-1)
#last_row = dataset_all.shape[0]-1
#dataset_all = dataset_all.drop(last_row, axis=0)  

#training_dates = Iteration.Iteration('1993-08-19', '2014-12-01')
#testing_dates  = Iteration.Iteration('2014-12-02', '2016-04-19')
#training_dates.calculate_indices(dataset)
#testing_dates.calculate_indices(dataset)
#
#trainDates = []
#testDates = []
#trainDates.append(training_dates.lowerIndex)
#trainDates.append(training_dates.upperIndex)
#testDates.append(testing_dates.lowerIndex)
#testDates.append(testing_dates.upperIndex)
#    
#trainX, trainY, testX, testY, cols = ml_dataset.dataset_to_train_using_dates(dataset, trainDates, testDates, binary=False, shiftFeatures=False, shiftTarget=False)

In [1]:
SEED = 42
colY = 'IBEX_RD_B10_Close'
colsDiff = 10
colsToShift = 1
#cols = [col for col in dataset.columns if 'HSI' in col or'N225' in col or 'AXJO' in col or 'IBEX' in col]
#dataset = dataset[cols]

df_x = dataset.filter(regex=('RD_B10_Close'))
last_row = list(range(df_x.shape[0] - colsToShift, df_x.shape[0] ))
df_x = df_x.drop(last_row, axis=0)
#df_x = df_x.drop(colY, axis=1)
        
df_y = dataset[colY].shift(-colsToShift)
last_row = list(range(df_y.shape[0] - colsToShift, df_y.shape[0] ))
df_y = df_y.drop(last_row, axis=0)

print 'Dataset shape %s' % str(dataset.shape)
print 'X shape %s' % str(df_x.shape)
print 'Y shape %s' % str(df_y.shape)

training_dates = Iteration.Iteration('2008-06-17', '2011-09-01')
testing_dates  = Iteration.Iteration('2012-09-04', '2014-10-06')
training_dates.calculate_indices(dataset)
testing_dates.calculate_indices(dataset)

trainDates = []
testDates = []
trainDates.append(training_dates.lowerIndex)
trainDates.append(training_dates.upperIndex)
testDates.append(testing_dates.lowerIndex)
testDates.append(testing_dates.upperIndex)

total = (trainDates[1]-trainDates[0]) + (testDates[1]-testDates[0])
tr = float(trainDates[1]-trainDates[0]) / total * 100.0
te = float(testDates[1]-testDates[0]) / total * 100.0

print "Training: from %s to %s" % (str(training_dates.startDate), str(training_dates.endDate))
print "Testing: from %s to %s" % (str(testing_dates.startDate), str(testing_dates.endDate))
print "%.3f %% training %.3f %% testing" % (tr,te)
print "%d training %d testing" % (trainDates[1]-trainDates[0], testDates[1]-testDates[0])        
trainX, trainY, testX, testY = ml_dataset.train_arrays_experiments(df_x, df_y, trainDates, testDates)
                
                
#svc = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=250)
svc = svm.LinearSVC()#kernel='poly')
#svc = discriminant_analysis.QuadraticDiscriminantAnalysis()
#svc = linear_model.LogisticRegression()
svc.fit(trainX, trainY)
y_pred = svc.predict(testX)
print "Score %s " % metrics.accuracy_score(testY, y_pred)

In [1]:
for i in range(testY.shape[0]):
    print "## TODAY - %s ##" % dataset_all.iloc[testing_dates.lowerIndex+i]['Date']
    print "Date %s \t Close %f \t Bin %d" % (dataset_all.iloc[testing_dates.lowerIndex-9+i]['Date'], dataset_all.iloc[testing_dates.lowerIndex-9+i]['IBEX_Close'],dataset_all.iloc[testing_dates.lowerIndex-9+i]['IBEX_RD_B10_Close']) 
    print "Date %s \t Close %f \t Bin %d" % (dataset_all.iloc[testing_dates.lowerIndex+i]['Date'], dataset_all.iloc[testing_dates.lowerIndex+i]['IBEX_Close'],dataset_all.iloc[testing_dates.lowerIndex+i]['IBEX_RD_B10_Close'])
    print "Date %s \t Close %f \t Bin %d" % (dataset_all.iloc[testing_dates.lowerIndex+1+i]['Date'], dataset_all.iloc[testing_dates.lowerIndex+1+i]['IBEX_Close'], dataset_all.iloc[testing_dates.lowerIndex+1+i]['IBEX_RD_B10_Close'])
    
    print "Test %d \t Pred %d" % (testY[i], y_pred[i])
    if testY[i] == 1:
        print "El índice SUBE de precio desde el día %s hasta el día %s" % (dataset_all.iloc[testing_dates.lowerIndex-9+i]['Date'],dataset_all.iloc[testing_dates.lowerIndex+1+i]['Date'])
        print "* BUY shares for day %s *" % dataset_all.iloc[testing_dates.lowerIndex+1+i]['Date']
        #Buy only if today`s price is lower than 10 days ago
    else:
        print "El índice BAJA de precio desde el día %s hasta el día %s" % (dataset_all.iloc[testing_dates.lowerIndex-9+i]['Date'],dataset_all.iloc[testing_dates.lowerIndex+1+i]['Date'])        
        print "* SELL shares for day %s *" % dataset_all.iloc[testing_dates.lowerIndex+1+i]['Date']

In [42]:
#data_manipulation.write_csv_data(GOLD, '/Users/Pablo/Desktop/TFM/Data/GOLD.csv')
#data_manipulation.write_csv_data(SILVER, '/Users/Pablo/Desktop/TFM/Data/SILVER.csv')
#data_manipulation.write_csv_data(PLAT, '/Users/Pablo/Desktop/TFM/Data/PLAT.csv')
#data_manipulation.write_csv_data(OIL_BRENT, '/Users/Pablo/Desktop/TFM/Data/OIL_BRENT.csv')
#
#data_manipulation.write_csv_data(USD_GBP, '/Users/Pablo/Desktop/TFM/Data/USD_GBP.csv')
#data_manipulation.write_csv_data(JPY_USD, '/Users/Pablo/Desktop/TFM/Data/JPY_USD.csv')
#data_manipulation.write_csv_data(AUD_USD, '/Users/Pablo/Desktop/TFM/Data/AUD_USD.csv')
#
#data_manipulation.write_csv_data(INDEX_DJIA, '/Users/Pablo/Desktop/TFM/Data/INDEX_DJIA.csv')
#data_manipulation.write_csv_data(INDEX_HSI, '/Users/Pablo/Desktop/TFM/Data/INDEX_HSI.csv')
#data_manipulation.write_csv_data(INDEX_IBEX, '/Users/Pablo/Desktop/TFM/Data/INDEX_IBEX.csv')
#data_manipulation.write_csv_data(INDEX_N225, '/Users/Pablo/Desktop/TFM/Data/INDEX_N225.csv')
#data_manipulation.write_csv_data(INDEX_SP500, '/Users/Pablo/Desktop/TFM/Data/INDEX_SP500.csv')
#data_manipulation.write_csv_data(INDEX_AXJO, '/Users/Pablo/Desktop/TFM/Data/INDEX_AXJO.csv')
#data_manipulation.write_csv_data(INDEX_FCHI, '/Users/Pablo/Desktop/TFM/Data/INDEX_FCHI.csv')
#data_manipulation.write_csv_data(INDEX_GDAXI, '/Users/Pablo/Desktop/TFM/Data/INDEX_GDAXI.csv')